# Objective: Uncover Trends & Patterns in Physician Quality

1. Read in the data

In [ ]:
import pandas as pd
import numpy as np       

In [20]:
mm = pd.read_csv('../data/raw/grp_public_reporting.csv', dtype='str')
print(mm.info())
mm.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193488 entries, 0 to 193487
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Facility Name         192502 non-null  object
 1    org_PAC_ID           193488 non-null  object
 2    ACO_ID_1             23816 non-null   object
 3    ACO_nm_1             23816 non-null   object
 4    ACO_ID_2             18 non-null      object
 5    ACO_nm_2             18 non-null      object
 6    measure_cd           193488 non-null  object
 7    measure_title        193488 non-null  object
 8    invs_msr             193488 non-null  object
 9    attestation_value    81992 non-null   object
 10   prf_rate             111496 non-null  object
 11   patient_count        111496 non-null  object
 12   star_value           80036 non-null   object
 13   five_star_benchmark  80036 non-null   object
 14   collection_type      111496 non-null  object
 15   CCXP_ind        

,Facility Name,org_PAC_ID,ACO_ID_1,ACO_nm_1,ACO_ID_2,ACO_nm_2,measure_cd,measure_title,invs_msr,attestation_value,prf_rate,patient_count,star_value,five_star_benchmark,collection_type,CCXP_ind
0,SPECTRUM FAMILY EYE CENTER OPTOMETRIC PA,0042268971,NaN,NaN,NaN,NaN,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN,NaN,Y
1,MOBILE MBS INC,0042357527,NaN,NaN,NaN,NaN,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN,NaN,Y
2,RETINA SAN DIEGO,0042521015,NaN,NaN,NaN,NaN,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN,NaN,Y
3,KARPIK AND RICE EYECARE PC,0042534281,NaN,NaN,NaN,NaN,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN,NaN,Y
4,INTERCARE MEDICAL CLINIC,0042549800,NaN,NaN,NaN,NaN,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN,NaN,Y


2. Clean the data (missing values, duplicates, outliers)

In [21]:
#get rid of spaces 
mm.columns = mm.columns.str.strip()

In [22]:
# missing values
mm = mm.drop(columns=['ACO_ID_1', 'ACO_nm_1','ACO_nm_2', 'ACO_ID_2', 'ACO_nm_2', 'collection_type', 'CCXP_ind'])

In [24]:
mm.drop_duplicates(inplace=True)

In [25]:
mm

,Facility Name,org_PAC_ID,measure_cd,measure_title,invs_msr,attestation_value,prf_rate,patient_count,star_value,five_star_benchmark
0,SPECTRUM FAMILY EYE CENTER OPTOMETRIC PA,0042268971,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN
1,MOBILE MBS INC,0042357527,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN
2,RETINA SAN DIEGO,0042521015,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN
3,KARPIK AND RICE EYECARE PC,0042534281,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN
4,INTERCARE MEDICAL CLINIC,0042549800,IA_GRP_AHE_1,Enhance Engagement of Medicaid and Other Under...,N,Y,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
193483,"FRESENIUS VASCULAR CARE AUGUSTA, LLC",8527227875,QCDR_GRP_RPAQIR13_overall,Rate of Timely Documentation Transmission to D...,N,NaN,92,3382,NaN,NaN
193484,MAKRISMD LLC,8820255847,QCDR_GRP_RPAQIR13_overall,Rate of Timely Documentation Transmission to D...,N,NaN,93,4776,NaN,NaN
193485,CONNECTICUT IMAGE GUIDED SURGERY PC,9234022328,QCDR_GRP_RPAQIR13_overall,Rate of Timely Documentation Transmission to D...,N,NaN,95,1319,NaN,NaN
193486,"AMERICAN ACCESS CARE OF NEW JERSEY, LLC",9234188194,QCDR_GRP_RPAQIR13_overall,Rate of Timely Documentation Transmission to D...,N,NaN,97,1079,NaN,NaN


3. Visualization of trends & patterns

In [42]:
### What are the most common measures submitted?
attested = mm[mm['attestation_value'] == 'Y']
attested['measure_title'].value_counts().reset_index().sort_values(by='count', ascending=False).head(50)

,measure_title,count
0,ONC Direct Review Attestation,5985
1,Security Risk Analysis,5970
2,Provide 24/7 Access to MIPS Eligible Clinician...,5058
3,ONC-ACB Surveillance Attestation,4000
4,Query of the Prescription Drug Monitoring Prog...,3854
5,High Priority Practices Guide of the Safety As...,3751
6,Electronic Case Reporting Exclusion,3507
7,Immunization Registry Reporting,3170
8,Immunization Registry Reporting Exclusion,2773
9,Health Information Exchange (HIE) Bi-Direction...,2728


In [37]:
mm['measure_title'].unique()

array(['Enhance Engagement of Medicaid and Other Underserved Populations',
       'Promote Use of Patient-Reported Outcome Tools',
       'MIPS Eligible Clinician Leadership in Clinical Trials or CBPR',
       'Provide Education Opportunities for New Clinicians',
       'Comprehensive Eye Exams',
       'Create and Implement an Anti-Racism Plan',
       'Implement Food Insecurity and Nutrition Risk Identification and Treatment Protocols',
       'Use of certified EHR to capture patient reported outcomes',
       'Use evidence-based decision aids to support shared decision-making.',
       'Engage Patients and Families to Guide Improvement in the System of Care',
       'Engagement of Patients, Family, and Caregivers in Developing a Plan of Care',
       'Promote Self-management in Usual Care',
       'Use group visits for common chronic conditions (e.g., diabetes).',
       'Improved Practices that Engage Patients Pre-Visit',
       'Integration of patient coaching practices between vi